# Demo for Barcelona

> ☝️ This notebook is prepared for working in local

---

In this tutorial, ...

## Setup



In [ ]:
#Inicializando el entorno
! ./setup.sh

## Context Info

Vamos a darle contexto al caso de uso:

In [ ]:

aws_profile_name = 'INCLUDE_YOUR_AWS_PROFILE_NAME'
aws_bedrock_assume_role = "INCLUDE_YOUR_AWS_BEDROCK_ASSUME_ROLE_ARN"

text_to_text_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
text_to_image_model_id = "amazon.titan-image-generator-v1"

reference_url = 'https://doit.com'
market_research_data = """
Red hats with large feathers is a trend in the Australian market
"""


## Fase 0: Capturar pantalla para branding
---

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from PIL import Image

image_name="target_snapshot.png"

def create_snapshot(url):
    # Configurar Selenium para usar Chrome
    driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

    # Acceder a la URL
    driver.get(url)

    # Dar tiempo a la página para que se cargue completamente
    driver.implicitly_wait(10)  # Espera implícita de 10 segundos

    # Tomar el screenshot y guardarlo
    driver.save_screenshot(image_name)

    # Cerrar el navegador
    driver.quit()

create_snapshot(reference_url)

screenshot = Image.open(image_name)

# Display
screenshot

## Fase 1: Crear CSS a partir de la captura
---
Vamos a tomar el screenshot y extraer a través de **Claude 3** con el **modo vision** los principales colores para utilizarlo en la CSS de la Landing.

In [ ]:
import os
import sys
import base64

from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage
from pathlib import Path

from utils import bedrock

os.environ["AWS_PROFILE"] = aws_profile_name
os.environ["BEDROCK_ASSUME_ROLE"] = aws_bedrock_assume_role  # E.g. "arn:aws:..."
model_id = text_to_text_model_id

pathCSS="target.css"

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

model = BedrockChat(
    client=boto3_bedrock,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

img1_path = Path(image_name)
img1_base64 = base64.b64encode(img1_path.read_bytes()).decode("utf-8")

messages = [
    HumanMessage(
        content = [
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{img1_base64}",
                },
            },
            {
                "type": "text",
                "text": """Extract the corporate image and colors from the image, please note that the color in texts cannot be white as , and create a snippet like this one:
                :root {
                    --textDark: rgba(38, 50, 56, 1);
                    --textMedium: rgba(38, 50, 56, 0.7);
                    --borderMedium: rgba(38, 50, 56, 0.2); 
                    --borderLight: rgba(38, 50, 56, 0.075);
                    --accent: #f4b500;
                    --textAccent: #e6ac00;
                    --light: rgba(38, 50, 56, 0.035);
                }
                """
            },
        ]
    )
]

# Abre el archivo donde deseas guardar el contenido
with open(pathCSS, 'a') as f:
    for chunk in model.stream(messages):
        # Imprime el contenido en la consola
        print(chunk.content, end="", flush=True)
        # Escribe el mismo contenido en el archivo
        f.write(chunk.content)
        # Asegura que el contenido se escribe inmediatamente en el archivo
        f.flush()
        # Opcionalmente, puedes usar sys.stdout.flush() para asegurar que la salida se muestre inmediatamente en la consola
        sys.stdout.flush()

## Fase 2: Actualizar CSS
---
Vamos a introducir la configuración de la CSS en el fichero adecuado de Hugobricks

In [ ]:
import re

def sustituir_variables(css_original_path, nuevas_variables_path):
    # Leer el contenido del archivo CSS original
    with open(css_original_path, 'r') as file:
        contenido_original = file.read()

    # Leer el archivo con las nuevas variables y sus valores
    with open(nuevas_variables_path, 'r') as file:
        nuevas_variables = file.read()

    # Extraer las variables y sus valores del archivo de nuevas variables
    variables_dict = {}
    for line in nuevas_variables.splitlines():
        match = re.search(r'(--\w+):\s*(.*);', line)
        if match:
            variable, valor = match.groups()
            variables_dict[variable] = valor

    # Sustituir los valores de las variables en el contenido original
    for variable, nuevo_valor in variables_dict.items():
        contenido_original = re.sub(rf'({variable}:\s*).+?;', rf'\1{nuevo_valor};', contenido_original)

    # Guardar los cambios en el archivo CSS original
    with open(css_original_path, 'w') as file:
        file.write(contenido_original)

    print("Las variables han sido sustituidas correctamente.")

# Rutas de los archivos (ajustalas según tu estructura de directorios)
css_original_path = 'hugobricks/static/css/style.css'
nuevas_variables_path = 'target.css'

# Llamar a la función para realizar la sustitución
sustituir_variables(css_original_path, nuevas_variables_path)


In [ ]:
! head -18 hugobricks/static/css/style.css | tail -14

## Fase 3: Generar el contenido de la propuesta de valor
---

En base al estudio de mercado vamos a generar el contenido de nuestra propuesta de valor

In [ ]:
import sys

from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from pathlib import Path

prompt_template = """
                Imagine we are launching a new product based of some new market research we have been doing
                [
                {market_research_data}
                ]
                we would like to make a landing page for testing the market, I would like you to help me to
                create the value proposal with all the info with the following hugobricks template:
                [
                {vanilla_template}
                ]
                This is an example of how it should look like the template once the data is populated.
                Notice that the sections brick_reviews and brick_cta have been removed from the template, hence you should do the same on your response:
                [
                {real_example}
                ]
                I don't need any explanation, I need you to create the same format but changed the data based on the information from the market research, invent whatever you need. Watch out for offensive content, illegal or any other language that could damage our reputation.
                All the call to action buttons, change the caption to whatever you want but they should point to /get-started/ as this is going to be one single page.
                Produce only the template without any further explanation, so that I can copy and paste the file.
                The brick_cta module you do not need to populate it.
                """

def create_prompt(prompt_template, market_research_data, vanilla_template_path, real_example_path):
    # Leer el contenido del primer archivo
    with open(vanilla_template_path, 'r') as vanilla_template_file:
        vanilla_template = vanilla_template_file.read()

    # Leer el contenido del segundo archivo
    with open(real_example_path, 'r') as real_example_file:
        real_example = real_example_file.read()

    prompt = PromptTemplate.from_template(prompt_template)
    prompt_text = prompt.format(market_research_data=market_research_data,
                                vanilla_template=vanilla_template,
                                real_example=real_example
                                )

    return prompt_text

# Rutas de los archivos de ejemplo
vanilla_template_path = 'index_template.md'
real_example_path = 'hugobricks/content/en/_index.md'
pathIndex = "target_index.md"

# Llamar a la función y pasar las rutas de los archivos como argumentos
prompt = create_prompt(prompt_template, market_research_data, vanilla_template_path, real_example_path)

messages = [
    HumanMessage(
        content = [
            {
                "type": "text",
                "text": prompt
            }
        ]
    )
]

# Abre el archivo donde deseas guardar el contenido
with open(pathIndex, 'a') as f:
    for chunk in model.stream(messages):
        # Imprime el contenido en la consola
        print(chunk.content, end="", flush=True)
        # Escribe el mismo contenido en el archivo
        f.write(chunk.content)
        # Asegura que el contenido se escribe inmediatamente en el archivo
        f.flush()
        # Opcionalmente, puedes usar sys.stdout.flush() para asegurar que la salida se muestre inmediatamente en la consola
        sys.stdout.flush()

In [ ]:
! cp -v target_index.md hugobricks/content/en/_index.md

## Fase 4: Crear los conceptos de las imagenes
---
Con Claude 3 vamos a generar los conceptos de las imágenes que luego le pediremos a Titan que genere.

NOTA: Estamos utilizando los mismos parametros del modelo para generar el texto, pero podríamos ser más creativos en algunas partes que en otras cambiando la temperatura, etc.

In [ ]:
image_concepts_json = "target_images_concepts.json"

prompt_template = """
                Act as an expert in Marketing and design. Create the image's descriptions that we will pass to a text-to-image model for creating
                the images to attach to our value proposal. This is the value proposal
                [
                {value_proposal_data}
                ]
                Include the prompt and the negative prompt (this are the things you want to avoid in the image generation). Please do not include text on the images
                For example:
                [
                prompt = "a beautiful lake surrounded by trees with a mountain range at the distance"
                negative_prompts = "poorly rendered, poor background details, poorly drawn mountains, disfigured mountain features"
                ]
                I am expeciting 4 images: 
                [
                    - One image for the logo
                    - One image for the intro section
                    - One image for the bimage2 section
                    - One image for the bimage section
                ]
                And return all the descriptions in a json format, be sure that the object is in the first position of an array, as follows: on
                [
                {{
                    "logo":{{
                        "filename": "logo.png",
                        "prompt": "prompt description for the logo generation",
                        "negative_prompts": "negative prompt description for the logo generation",
                    }},
                    "intro":{{
                        "filename": "intro.png",
                        "prompt": "prompt description for the intro generation",
                        "negative_prompts": "negative prompt description for the intro generation",
                    }},
                    "bimage2":{{
                        "filename": "bimage2.png",
                        "prompt": "prompt description for the bimage2 generation",
                        "negative_prompts": "negative prompt description for the bimage2 generation",
                    }},
                    "bimage":{{
                        "filename": "baimage.png",
                        "prompt": "prompt description for the bimage generation",
                        "negative_prompts": "negative prompt description for the bimage generation",
                    }}
                }}
                ]
                I don't need any explanation, I need you to create the same format but changed the data based on the information from the value proposal, invent whatever you need. Watch out for offensive content, illegal or any other language that could damage our reputation.
                Produce only the json without any further explanation, so that I can copy and paste the file.
                """

def create_prompt(prompt_template, value_proposal_path):
    # Leer el contenido del primer archivo
    with open(value_proposal_path, 'r') as value_proposal_file:
        value_proposal = value_proposal_file.read()

    prompt = PromptTemplate.from_template(prompt_template)
    prompt_text = prompt.format(value_proposal_data=value_proposal)

    return prompt_text

value_proposal_path = pathIndex

prompt = create_prompt(prompt_template, value_proposal_path)

messages = [
    HumanMessage(
        content = [
            {
                "type": "text",
                "text": prompt
            }
        ]
    )
]

# Abre el archivo donde deseas guardar el contenido
with open(image_concepts_json, 'a') as f:
    for chunk in model.stream(messages):
        # Imprime el contenido en la consola
        print(chunk.content, end="", flush=True)
        # Escribe el mismo contenido en el archivo
        f.write(chunk.content)
        # Asegura que el contenido se escribe inmediatamente en el archivo
        f.flush()
        # Opcionalmente, puedes usar sys.stdout.flush() para asegurar que la salida se muestre inmediatamente en la consola
        sys.stdout.flush()

Fase 5: Crear las images basadas en los conceptos
---

Vamos a utilizar Amazon Titan como modelo de text-to-image

In [ ]:
import base64
import io
import json

images_dir= "demo_images"

# Leer y cargar el contenido del archivo JSON
with open(image_concepts_json, 'r') as file:
    images_json = json.load(file)

def create_body(prompt, negative_prompts):
    body = json.dumps(
    {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt,                    # Required
            "negativeText": negative_prompts   # Optional
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,   # Range: 1 to 5 
            "quality": "standard",  # Options: standard or premium
            "height": 1024,        # Supported height list in the docs 
            "width": 1024,         # Supported width list in the docs
            "cfgScale": 7.5,       # Range: 1.0 (exclusive) to 10.0
            "seed": 42             # Range: 0 to 214783647
        }
    }
    )
    return body

# Función para hacer una llamada a la API
def create_image(seccion, data):
    
    body = create_body(data['prompt'],data['negative_prompts'])

    #print(body)
    # Make model request
    response = boto3_bedrock.invoke_model(
        body=body,
        modelId=text_to_image_model_id,
        accept="application/json", 
        contentType="application/json"
    )

    # Process the image
    response_body = json.loads(response.get("body").read())
    img1_b64 = response_body["images"][0]

    os.makedirs(images_dir, exist_ok=True)

    # Decode + save
    img1 = Image.open(
        io.BytesIO(
            base64.decodebytes(
                bytes(img1_b64, "utf-8")
            )
        )
    )
    img1.save(f"{images_dir}/{data['filename']}")
    img1
    

# Iterar sobre cada objeto en el JSON
for object in images_json:
    # Iterar sobre cada sección dentro del objeto
    for image in object:
        create_image(image, object[image])



In [ ]:
%%sh
mkdir -p hugobricks/static/uploads/illustrations/demo/
cp demo_images/* hugobricks/static/uploads/illustrations/demo/


In [ ]:
import subprocess

executable_directory = "hugobricks"
executable_name = "hugo"
arguments = ["server"]
process = subprocess.Popen([executable_name] + arguments, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=executable_directory)
process_id = process.pid
print(f"Background process started with PID: {process_id} from directory {executable_directory}")


In [ ]:
import re, time

time.sleep(2)
command = ["lsof", "-nP", "-i4TCP", "-a", f"-p{process_id}"]
lsof_output = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = lsof_output.communicate()
output = stdout.decode('utf-8')

if output:
    print("Open ports by the process:")
    print(output)
    pattern = r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}:[0-9]+\b'
    match = re.search(pattern, output)
    if match:
        ip_port = match.group()
        print("http://" + ip_port)
    else:
        print("No IP address and port found in the output.")
else:
    print("No open ports found for the process or unable to retrieve information.")

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

Clean up
---

Parando Hugo Server

In [ ]:
import os
import signal

from IPython.display import display
import ipywidgets as widgets

def confirm_action(b):
    print("User confirmed. Proceeding with the destructive action...")
    os.kill(process_id, signal.SIGTERM)
    print(f"Background process with PID {process_id} has been terminated.")
    confirm_button.disabled = True
    deny_button.disabled = True

def deny_action(b):
    print("User did not confirm. Hugo Server is still running.")
    confirm_button.disabled = True
    deny_button.disabled = True


message_label = widgets.Label("Do you want to stop the Hugo Server?")

confirm_button = widgets.Button(description="Confirm")
deny_button = widgets.Button(description="Deny")

confirm_button.on_click(confirm_action)
deny_button.on_click(deny_action)

display(widgets.VBox([message_label, widgets.HBox([confirm_button, deny_button])]))

